In [23]:
import numpy as np
import os
from skimage.io import imread
from PIL import Image

names = os.listdir('male')
names.pop(0) #remove '.DS_Store' file

samples = []

for n in names:
    for j in range(1,2): # read only one face of each person
        file = './male/%s/%s.%d.jpg' % (n, n, j)
        samples.append(imread(file, as_grey=True))

samples = np.resize(np.array(samples), (len(samples), 36000))

In [72]:
# Compute the mean face

mean_face = np.zeros(36000)

for i in range(0, len(samples)):
    mean_face += samples[i]

mean_face /= len(samples)

def show(a):
    Image.fromarray(np.resize(a, (200, 180)) * 255).show()
    
show(mean_face)

In [91]:
# Compute difference faces
difference_faces = samples - mean_face 
    
# Covariance matrix of difference faces
cov = np.dot(difference_faces, difference_faces.T)

# Eigenvectors and eigenvalues of the covariance matrix
eigenface_val, eigenface_vec = np.linalg.eig(cov)

In [129]:
eigenfaces = np.dot(difference_faces.T, eigenface_vec).T

# Normalize eigenfaces (make them unit vectors)
for i in range(len(eigenfaces)):
    norm = np.linalg.norm(eigenfaces[i])
    eigenfaces[i] /= norm

take_eigenfaces = len(eigenfaces)

coeff = np.dot(eigenfaces, difference_faces.T)

def reconstruct(face, eigf=len(eigenfaces)):
    return (mean_face + np.dot(eigenfaces[:eigf].T, coeff[:eigf]).T[face])

show(reconstruct(0, 90))

In [133]:
for celeb in ['trump', 'hitler', 'guy0']:
    celeb = np.resize(imread('%s.jpg' % (celeb), as_grey=True), (1, 36000))

    diff = celeb - mean_face
    coef = np.dot(eigenfaces, diff.T)

    show(mean_face + np.dot(eigenfaces.T, coef).T)